In [3]:
from pymongo import MongoClient
import requests
import json
import pandas as pd
import numpy as np
from getpass import getpass
import os

import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [4]:
token = getpass()

········


# Acces a db collection

In [56]:
def access_mdb_local_collection (database, collection):

    """ Function that returns a collection inside a DataBase 
    host in my local Server """

    # 1. Connect to my local Server
    client = MongoClient("localhost:27017")
    
    # 2. Access a DataBase:
    db = client[database]

    # 3. Acces a collection inside the given DataBase
    c = db.get_collection(collection)

    # 4. Return collection into a given DataBase host in a given Server
    return c

## Extract from FourSquare API

In [57]:
def foursquare_places(location, radius, query="" ,category = ""):
    
    """
    Search for places by distance on a circular boundary in ForeSquare API and returns a .json()
    Optional filter, venue and/or category can be added
    Response limited to 50 (maximum)

    :location: str : "latidude,longitude" around which to retrieve place information  eg. "43.37012643,-8,39114853"
    :radius: int : radius in meters.define the area to bias search results.  eg. 10000
    :query: str : OPTIONAL:  a string to be matched against all content for this place
    :category: int: OPTIONAL : returns FSQ Places matching the specified id categories
       see category taxonomy -->  https://location.foursquare.com/places/docs/categories

    """


    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "query": query,
        "ll": location,
        "radius":radius,
        "categories":category,
        "limit":50,
        "sort":"DISTANCE"  #Por defecto relevancia. 
    }

    headers = {
        "Accept": "application/json",
        "Authorization": token
    }

    return requests.get(url, params=params, headers=headers).json()

## Insert into a given collection:

In [58]:
def insert_unique_parents(response, mdb_collection):
    
    """
    Inserts the results of a reques that ARE NOT in the 
    given collection AND ARE NOT related to other object(childrens)

    response: --- : what to insert
    collection: ----: where to insert it
    """
    
    # 1. set conter for inserted objects
    inserted = 0
    
    # 3. Get the list of all elements alrready registered in the collection
    registered = [i['fsq_id'] for i in mdb_collection.find({}, {"_id":0,"fsq_id":1})]
    
    # 4. Iterate trough all elements of the response 
    for i in response["results"]:
        
        #Need to meet 2 conditions to be inserted:
        # It is not alrready registered:
        cond1 = i['fsq_id'] not in registered
        
        ## It's not a children of a parent:
        cond2 = "parent" not in list(i["related_places"].keys())
        
        # Check if the conditions are meet:
        if cond1 and cond2 == True:
            mdb_collection.insert_one(i)
            inserted += 1
            
    print(f'{inserted} items inserted into the collection')
               
    return

## Visualization Functions:

Aqui la funcion cabia con respecto al anterior

In [59]:
def city_places(city, collection):
    
    c = access_mdb_local_collection ("foursquare", collection)
    
    # 1. Set the filter to the objects --> REVIEW option just locality??¿?¿ lo de Valencia - comunidad valencia 
    filt = {}
    
    # 2. Select the filds that we want to target:
    projection = { "_id":0,"categories":1, "geocodes":1,"name":1}
    
    # 3. Extract data
    datos = list(c.find(filt,projection))
    
    # 4. Generate an empty dictionary with the categories to save:
    register = {"city": [], "place_name":[], "lable":[],"lat":[], "lon":[]}
    
    # 5. Iterate trowh all elements found:
    for i in datos:

        info = extract_parameters(i) # info it's a dictionary 
            # example --> info = {"name": name, "cat_code":cat_code, "lat":lat, "lon":lon}
            
            
        name = info["name"]
        lable =info["cat_code"]
        lat = info["lat"]
        lon =  info["lon"]
        
        register["city"].append(city)
        register["place_name"].append(name)
        register["lable"].append(lable)
        register["lat"].append(lat)
        register["lon"].append(lon)

        df = pd.DataFrame(register)
        
    return df

In [60]:
def extract_parameters(i):
            
            name = i["name"]
            
            lat = i["geocodes"]["main"]["latitude"]
            lon = i["geocodes"]["main"]["longitude"]
            
            # as elements can have more than one categorie
            #we need to make sure we use the ones that we
            #have consider in markers. We know they all have
            #at least one of them because we extract the 
            #info from the API using categorie filter:
            
            
            cat_list = [x["id"] for x in i["categories"]]
            cat_code = [key for key in markers.keys() if key in cat_list][0]
              
            """
            The same before comprehension list:
            
            for key in markers.keys():
                if key in cat_list
                    cat_code = key
                    
            With the comprehension list, we need to acces the first 
            element -> we know there will be just one.
            """
            params = {"name": name, "cat_code":cat_code, "lat":lat, "lon":lon}
            
            return params

In [61]:
def icon_set(category_code): # marker of categorye [10051]
    
    icon =  Icon(
            color = markers[category_code][1], #red
            icon_color = markers[category_code][2], #black
            icon = markers[category_code][3], #dog
            prefix="fa" # font-awesome website: fa
                )
            
    return icon

In [62]:
def city_markers(df, to_map):
    
        for index, row in df.iterrows():
            
            #What kind of place it is: (code)
            category_code = df.iloc[index]["lable"]
            
            #Set the icon for that kind of place:
            icon_conf = icon_set(category_code)
            
            #Extract position in witch locates:
            lat = df.iloc[index]["lat"]
            lon = df.iloc[index]["lon"]
            coordenates = [lat, lon]
            
            #Retrieve the category_code name (ej:"stadium")
            # we know the position in the dictionary
            lable = markers[category_code][0]
            
            #Set the marker:
            marker = Marker(coordenates, tooltip=lable ,  icon=icon_conf)
            
            marker.add_to(to_map)
        
        return

In [63]:
def heat_map(df, to_map):
    
    HeatMap(data = df[["lat", "lon"]]).add_to(to_map)
        
    return

# MAIN:

### Extraer Datos de las ciudades seleccionadas:

In [64]:
Posibles = {
    "Madrid" : "40.40841191,-3.68760088",
    "Barcelona" : "41.38424664,2.17634927",
    "Valencia" : "39.47534441,-0.37565717",
    "Sevilla" : "37.38620512,-5.99251368"   
}

### MADRID

In [65]:
location_dict = {"Madrid" : "40.40841191,-3.68760088"}

1. Extract From API, Insert into MG

In [66]:
categories = {
    "stadium" : 10051,
    "dog_park" : 16033,
    "vegan_rest" : 13377,
    #"nigth_club" : 10032
}

#Para cada categoria:
for key, cat in categories.items():
    
    location = location_dict["Madrid"]
    radius = 10000 #(10km)
    query = ""
    category = cat
        
    #print(f'----Location {key}, category {key2}----')
    
    # EXTRACT 
    response = foursquare_places(location, radius, query ,category)
    
    #CONCET
    mdb_collection = access_mdb_local_collection ("foursquare", "madrid")
    
    #INSERT:
    insert_unique_parents(response, mdb_collection)
    
    #desable printing
    #print(f'---------------------------------------')

0 items inserted into the collection
0 items inserted into the collection
0 items inserted into the collection


2. Extract from Mongo and Create a DF

In [67]:
markers = {
    10051: ["stadium" , "orange", "black", "baseball"],
    16033: ["dog_park", "green", "white", "dog"],
    13377: ["vegan_rest", "white", "green", "leaf"],
    #10032: ["nigth_club", "black", "white", "champagne-glasses"]
}

In [68]:
df = city_places("madrid", "madrid")

In [69]:
# Create the map 
city_map = Map([40.40841191,-3.68760088], zoom_start=13,  tiles='CartoDB positron')

#Add the markers 
city_markers(df, city_map)

city_map.save(f"{location}_markmap.html")

city_map

In [71]:
#map and dataframe
city_map =  Map([40.40841191,-3.68760088], zoom_start=13,  tiles='CartoDB positron')
heat_map(df, city_map)
city_map.save(f"{location}_heatmap.html")
city_map

In [34]:
Circle(
    location= [40.40841191,-3.68760088],
    radius=1000,
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.3,
    tooltip=f'Radio:m'
).add_to(city_map)

### VALENCIA

In [72]:
location_dict = {"Valencia" : "39.47534441,-0.37565717"}

1. Extract From API, Insert into MG

In [73]:
categories = {
    "stadium" : 10051,
    "dog_park" : 16033,
    "vegan_rest" : 13377,
    #"nigth_club" : 10032
}

#Para cada categoria:
for key, cat in categories.items():
    
    location = location_dict["Valencia"]
    radius = 10000 #(10km)
    query = ""
    category = cat
        
    #print(f'----Location {key}, category {key2}----')
    
    # EXTRACT 
    response = foursquare_places(location, radius, query ,category)
    
    #CONCET
    mdb_collection = access_mdb_local_collection ("foursquare", "valencia")
    
    #INSERT:
    insert_unique_parents(response, mdb_collection)
    
    #desable printing
    #print(f'---------------------------------------')

0 items inserted into the collection
0 items inserted into the collection
0 items inserted into the collection


2. Extract from Mongo and Create a DF

In [74]:
markers = {
    10051: ["stadium" , "orange", "black", "baseball"],
    16033: ["dog_park", "green", "white", "dog"],
    13377: ["vegan_rest", "white", "green", "leaf"],
    #10032: ["nigth_club", "black", "white", "champagne-glasses"]
}

In [75]:
df = city_places("valencia", "valencia")

In [76]:
# Create the map 
city_map = Map([39.47534441,-0.37565717], zoom_start=13,  tiles='CartoDB positron')

#Add the markers 
city_markers(df, city_map)

city_map.save(f"{location}_markmap.html")

city_map

In [78]:
#map and dataframe
city_map =  Map([39.47534441,-0.37565717], zoom_start=13,  tiles='CartoDB positron')
heat_map(df, city_map)
city_map.save(f"{location}_heatmap.html")
city_map